In [22]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"

model_name="Local-Novel-LLM-project/Vecteus-v1"
model_name="hatakeyama-llm-team/tanuki_inst_0515test"
model_name="kanhatakeyama/0516tanuki_lr5e5_epoch1"
model_name="hatakeyama-llm-team/0524clean_halcination_little_codes_step1600"
#model_name="../X_merge/merged_models/0517"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)


In [23]:
t_list=[
"1+1=",
"1+2+3-4=",
"4/4+1.5=",
"1*23.5-4563+(-4)=",
]
for t in t_list:
    print(tokenizer.tokenize(t,add_special_tokens=False))

['▁', '1', '+', '1', '=']
['▁', '1', '+', '2', '+', '3', '-', '4', '=']
['▁', '4', '/', '4', '+', '1', '.', '5', '=']
['▁', '1', '*', '2', '3', '.', '5', '-', '4', '5', '6', '3', '+', '(', '-', '4', ')=']


In [24]:
o='1. "The Great Gatsby"（20世紀フォックス・フィルムズ/ワーナーブラザーズ共同制作 - 映画『華麗なるギャツビー』として公開された作品 '
len(tokenizer.tokenize(o,add_special_tokens=False))

39

In [25]:

tokens=[5092, 272, 1045, 2850, 327, 8,11267, 269, 564, 298, 2]
tokenizer.decode(tokens,)

'### 応答:\n はい、可能です</s>'

In [26]:
t_list=[
"\n\n### 応答:\n",
"\n\n### 応答:\n「",
"\n\n### 応答:\nああ",
"\n\n### 応答:\naaa",
"1\n\n### 応答:\naaa",
"あ\n\n### 応答:\naaa",
"」\n\n### 応答:\naaa",
]
t_list=[
    "",
    "a \n a",
"a\n   ",
"\n\n### 指示:\n",
"\n\n### 指示:\n「",
"\n\n### 指示:\nああ",
"\n\n### 指示:\naaa",
"1\n\n### 指示:\naaa",
"あ\n\n### 指示:\naaa",
"」\n\n### 指示:\naaa",
]
for t in t_list:
    enc=tokenizer.encode(t)
    print(enc,tokenizer.decode(enc))

tokenizer.decode(enc)

[2] </s>
[355, 272, 8, 355, 2] a 
 a</s>
[355, 8, 3223, 2] a
   </s>
[272, 8, 8, 48979, 272, 3994, 327, 8, 2] 

### 指示:
</s>
[272, 8, 8, 48979, 272, 3994, 327, 8, 292, 2] 

### 指示:
「</s>
[272, 8, 8, 48979, 272, 3994, 327, 8, 2721, 2] 

### 指示:
ああ</s>
[272, 8, 8, 48979, 272, 3994, 327, 8, 55841, 2] 

### 指示:
aaa</s>
[272, 279, 8, 8, 48979, 272, 3994, 327, 8, 55841, 2] 1

### 指示:
aaa</s>
[4543, 8, 8, 48979, 272, 3994, 327, 8, 55841, 2] あ

### 指示:
aaa</s>
[272, 285, 8, 8, 48979, 272, 3994, 327, 8, 55841, 2] 」

### 指示:
aaa</s>


'」\n\n### 指示:\naaa</s>'

In [27]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16,
                                            )


In [ ]:
model.config.repetition_penalty=1.2

In [ ]:
prompt="""以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
### 指示:
将来的な映画製作者が学ぶべき五つの受賞歴のあるドキュメンタリー映画とそれぞれの背景説明を提案してください。
### 応答:
"""
with torch.no_grad():
    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    output_ids = model.generate(input_ids.to(model.device), max_length=250, pad_token_id=tokenizer.pad_token_id)

print(tokenizer.decode(output_ids.tolist()[0]))

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。
### 指示:
将来的な映画製作者が学ぶべき五つの受賞歴のあるドキュメンタリー映画とそれぞれの背景説明を提案してください。
### 応答:
1. "The Greatest Show" - この作品では、アメリカ合衆国で最も偉大だったエンターテイナーであるマイケル・ジャクソンについて取り上げられています。このドキュメンタリーは20世紀75年代から89年までの間に行われた彼の活動や生活環境など様々なことを紹介しており、その時代において非常に大きな人気を得ました。</s>


In [ ]:
#model.load_adapter(model_path)

In [ ]:
#層の表示
for name, param in model.named_parameters():
    print(name)
    if "gate" in name:
        print("***",name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
*** model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
*** model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
*** mode

In [ ]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=250, 
              repetition_penalty=1.15,
              temperature=0.6,
              #repetition_penalty=1.,
              #temperature=0.1,
              #top_p=1.0,
              #top_k=0.
              )


In [ ]:
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"
answer_template="\n\n### 応答:\n"

def gen_prompt(q):
    return f"{question_template}{q}{answer_template}"
    return question_template+q+answer_template

gen_prompt("こんにちは")

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nこんにちは\n\n### 応答:\n'

In [ ]:
questions=[
"こんにちは",
"元気ですか?",
"いい天気ですね",
"明日の天気は?",
"計算してください 1+1はいくつですか｡",
"フィボナッチ数列を生成するpythonのコードを作成してください",
"ドラえもんの友達はだれですか?",
"レイリー散乱とはなんですか",
"hello!",
"今の天気は晴れです｡この結果をjsonで出力してください｡ ",
"純粋理性批判はたぬきに理解できますか?",
]
questions=[
"たぬきに純粋理性批判は理解できますか?",
"フィボナッチ数列を生成するpythonのcode",
"元気ですか?",
 "日本の首相は?",
"東京科学大学の学長は?",
 "東京科学大学とは?"   ,
"次の四則演算をしなさい\n 1+2+3",
"1+2+3-4はいくつか",
"1+2+3",
"1/2+3を計算せよ",
"計算をせよ: 0.4+1",
"将来的な映画製作者が学ぶべき五つの受賞歴のあるドキュメンタリー映画とそれぞれの背景説明を提案してください。",
]
#questions=[
#]


for question in questions:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    res=pipe(inp,max_new_tokens=256)[0]["generated_text"][len(inp):]
    print(res)
    #print(tokenizer.encode(res)[:4])

--------------------
将来的な映画製作者が学ぶべき五つの受賞歴のあるドキュメンタリー映画とそれぞれの背景説明を提案してください。
1. "The Greatest Show" - この作品では、アメリカ合衆国で最も偉大なショーとして評価されており、エミー賞やトニー賞など多くの栄誉を得ています。このドキュメンタリーは、その成功までの道のりについて詳細に取り上げます。


In [21]:
inp

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n将来的な映画製作者が学ぶべき五つの受賞歴のあるドキュメンタリー映画とそれぞれの背景説明を提案してください。\n\n### 応答:\n'

In [ ]:
#tokenizer.piece_size = 100000
#tokenizer.add_special_tokens({'additional_special_tokens': ['\\n']})
#tokenizer.add_prefix_space=False
inp="フィボナッチ数列を生成するpythonのコード"

inp=gen_prompt("フィボナッチ数列を生成するpythonのコードを作成してください")
inp=gen_prompt("フィボナッチ数列を生成するCの再帰コードを作成してください")
res=pipe(inp,max_new_tokens=512)[0]["generated_text"]
res_cut=res[len(inp):]
#tokenizer.encode("### 応答:\n はい、可能です")
print(inp)
print(res_cut)
tokenizer.encode(res)[:4]

In [ ]:
res
tokenizer.encode(res)[:4]
res

# ベンチマーク問題での応答評価

In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=300, 
              repetition_penalty=1.2,
              temperature=0.1,
              top_p=1.0,
              top_k=0.
              )

In [ ]:
#ベンチマーク問題への応答性の軽い評価
import pandas as pd
jaster_path="bench_data/jaster.txt"
with open(jaster_path,"r") as f:
    input_list=f.readlines()

for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])

In [ ]:
import random
jmt_path="bench_data/jmt.csv"
df=pd.read_csv(jmt_path)
input_list=df["input"].to_list()
random.shuffle(input_list)
for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])

In [ ]:

pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=300, 
              #repetition_penalty=1.,
              temperature=0.1,
              #top_p=1.0,
              #top_k=0.,
              do_sample=True,
              )

question="""日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。"""
inp=gen_prompt(question)
r=pipe(inp)[0]["generated_text"][len(inp):]
print(r)

In [ ]:
prompt=inp
do_sample=True
#do_sample=False
temperature=0.1
max_new_token=30

question="""日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。"""
#question='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:\n'
#question='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:'
inp=gen_prompt(question)

inp='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:'
def ask(prompt):
    input_ids = tokenizer([prompt]).input_ids
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        do_sample=do_sample,
        temperature=temperature,
        max_new_tokens=max_new_token,
        #repetition_penalty=1.0,
    )


    output_ids_=output_ids[0][len(input_ids[0]) :]
    output_text = tokenizer.decode(output_ids_, skip_special_tokens=True)
    return output_text

for i in range(3):
    print(i)
    print(ask(inp))